# 서포트 벡터 머신(Support Vector Machines)

- 회귀, 분류, 이상치 탐지 등에 사용되는 지도학습 방법
- 클래스 사이의 경계에 위치한 데이터 포인트를 서포트 벡터(support vector)라고 한다.
- 각 지지 벡터가 클래스 사이의 결정 경계를 구분하는데 얼마나 중요한지를 학습
- 각 지지 벡터 사이의 마진이 가장 큰 방향으로 학습
- 지지 벡터 까지의 거리와 지지 벡터의 중요도를 기반으로 예측을 수행

![support vector machine](https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Svm_separating_hyperplanes.png/220px-Svm_separating_hyperplanes.png)

- H3은 두 클래스의 점들을 제대로 분류하고 있지 않음
- H1과 H2는 두 클래스의 점들을 분류하는데, H2가 H1보다 더 큰 마진을 갖고 분류하는 것을 확인할 수 있음

In [1]:
import multiprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.svm import SVR, SVC 
from sklearn.datasets import load_diabetes
from sklearn.datasets import load_breast_cancer, load_iris, load_wine
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.manifold import TSNE

## SVM을 이용한 회귀 모델과 분류 모델

### SVM을 사용한 회귀 모델 (SVR)

In [4]:
# url 을 통해서 불러오세요 
import pandas as pd
url = "https://raw.githubusercontent.com/myoh0623/dataset/main/boston.csv"
boston_df = pd.read_csv(url)
x = boston_df.iloc[:, 0:-1]
y = boston_df.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2)


model = SVR()
model.fit(x_train, y_train)

# 회귀 모델이라 score 가 낮을수 있다. 
print(f'Train Data Score: {model.score(x_train, y_train)}')
print(f'Test Data Score: {model.score(x_test, y_test)}')

Train Data Score: 0.21537081803194957
Test Data Score: 0.23908939655470351


### SVM을 사용한 분류 모델 (SVC)

In [5]:
x, y = load_breast_cancer(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2)

model = SVC()
model.fit(x_train, y_train)

model.score(x_train, y_train), model.score(x_test, y_test)

(0.9186813186813186, 0.9298245614035088)

In [7]:
# 실습 C-value를 변경하면서 학습을 시켜 보세요
# f1-score, recall 를 구해보세요
# 0.01, 0.1, 1, 10
# 질병의 예방에 있어서는 precision 이 중요하고
# 질병을 판단 하는데 있어서는 recall 이 중요하다.
from sklearn.metrics import f1_score, recall_score
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
models = [SVC(C=0.01), SVC(C=0.1), SVC(C=1), SVC(C=10)]

for model in models:
    model.fit(x_train, y_train)
    print(f"train data score : {model.score(x_train, y_train)}")
    print(f"test data score : {model.score(x_test, y_test)}")
    print(f"f1-score score : {f1_score(y_test, model.predict(x_test))}")
    print(f"recall-score score : {recall_score(y_test, model.predict(x_test))}")
    print(" ")

train data score : 0.7868131868131868
test data score : 0.8508771929824561
f1-score score : 0.903954802259887
recall-score score : 1.0
 
train data score : 0.8879120879120879
test data score : 0.9122807017543859
f1-score score : 0.9411764705882353
recall-score score : 1.0
 
train data score : 0.9186813186813186
test data score : 0.9298245614035088
f1-score score : 0.9518072289156626
recall-score score : 0.9875
 
train data score : 0.9230769230769231
test data score : 0.9298245614035088
f1-score score : 0.9506172839506173
recall-score score : 0.9625
 


## 커널 기법

- 입력 데이터를 고차원 공간에 사상해서 비선형 특징을 학습할 수 있도록 확장하는 방법
- scikit-learn에서는 Linear, Polynomial, RBF(Radial Basis Function)등 다양한 커널 기법을 지원
- https://scikit-learn.org/stable/modules/svm.html#kernel-functions

In [11]:
# load_boston 보스턴 집값

models = [SVR(kernel = "linear"), SVR(kernel = "rbf"), SVR(kernel = "poly")]

# 회귀 모델이라 score 가 낮을수 있다. 
for model in models:
    model.fit(x_train, y_train)
    print(f'Train Data Score: {model.score(x_train, y_train)}')
    print(f'Test Data Score: {model.score(x_test, y_test)}')

Train Data Score: 0.4554141029558435
Test Data Score: 0.6344363976459313
Train Data Score: 0.7411306673945179
Test Data Score: 0.7248207987630981
Train Data Score: 0.5326027725301978
Test Data Score: 0.5724962454050346


In [ ]:
# 유방암 데이터에 컬널별 분류를 적용해 보자.
# ["linear", "poly", "rbf", "sigmoid"]
# https://scikit-learn.org/stable/modules/svm.html#kernel-functions

## 매개변수 튜닝

- SVM은 사용하는 커널에 따라 다양한 매개변수 설정 가능
- 매개변수를 변경하면서 성능변화를 관찰

In [12]:
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.01, 0.1, 1, 10]
}


grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(x_train, y_train)

print(f'Best parameters: {grid_search.best_params_}')
print(f'Best cross-validation score: {grid_search.best_score_}')

best_model = grid_search.best_estimator_


y_pred = best_model.predict(x_test)


print(f'Test set score: {best_model.score(x_test, y_test)}')
print(f'F1-score: {f1_score(y_test, y_pred)}')
print(f'Recall: {recall_score(y_test, y_pred)}')

Best parameters: {'C': 10, 'kernel': 'linear'}
Best cross-validation score: 0.9516483516483516
Test set score: 0.9473684210526315
F1-score: 0.9634146341463414
Recall: 0.9875


## 데이터 전처리

- SVM은 입력 데이터가 정규화 되어야 좋은 성능을 보임
- 주로 모든 특성 값을 [0, 1] 범위로 맞추는 방법을 사용
- scikit-learn의 StandardScaler 또는 MinMaxScaler를 사용해 정규화

In [13]:
# StandardScaler, MinMaxScaler를 sklearn 의 docs 에서 확인 
# load_breast_cancer 에 StandardScaler 를 이용해 정규화를 진행 하고 학습을 시켜 보세요 

In [15]:
from sklearn.preprocessing import StandardScaler

In [18]:
x, y = load_breast_cancer(return_X_y = True)
scaler = StandardScaler()
x = scaler.fit_transform(x) # fit + transform
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1234)

In [19]:
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.01, 0.1, 1, 10]
}

grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(x_train, y_train)

print(f'Best parameters: {grid_search.best_params_}')
print(f'Best cross-validation score: {grid_search.best_score_}')

best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)
print(f'Test set score: {best_model.score(x_test, y_test)}')
print(f'F1-score: {f1_score(y_test, y_pred)}')
print(f'Recall: {recall_score(y_test, y_pred)}')

Best parameters: {'C': 0.1, 'kernel': 'linear'}
Best cross-validation score: 0.9788782489740082
Test set score: 0.958041958041958
F1-score: 0.967032967032967
Recall: 1.0
